# ABOUT:
- this notebook is a test run of SimpleX using RecBole API

# run 0 - notes:
- the actual interaction data in under .inter_feat attribute
- it appears that the value of "by" hyperparameter is the number of negative samples per user. 
    - But why can't i change this value to other than 1?
    - maybe i need to set "by" such that (1+by) is a divisible by the batch size? other its set back to 1?
- for each positive item, the corresponding negative items are in the same batch
- update: number of negative samples is controlled by "neg_sampling" param not "train_neg_sample_args"

In [11]:
model = "SimpleX"
dataset = 'ml-100k'
config_file_list = ['default_config.yaml', 'main_config.yaml']
config_dict = {
    "show_progress": False,
    "log_wandb":True,
    "eval_step": 10,
    "epochs": 50, 
    "eval_args": {"split": {"RS": [6,2,2]}},
    "valid_metric":"NDCG@20",
    "metrics":["Recall","NDCG"],
    "topk":[10,20],
    
    "embedding_size":32,
    "margin":0.8,
    "negative_weight":100,
    "gamma":0.5,
    "reg_weight":0,
    "aggregator":"mean",
    "history_len":10,
    "neg_sampling":{"uniform":100}
}
saved = True

In [12]:
import argparse

In [13]:
"""
recbole.quick_start
########################
"""
import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color


# def run_recbole(model=None, dataset=None, config_file_list=None, config_dict=None, saved=True):

# configurations initialization
config = Config(model=model, dataset=dataset, config_file_list=config_file_list, config_dict=config_dict)
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)
logger = getLogger()

logger.info(config)

# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

# # model loading and initialization
# init_seed(config['seed'], config['reproducibility'])
# model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
# logger.info(model)

# # trainer loading and initialization
# trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# # model training
# best_valid_score, best_valid_result = trainer.fit(
#     train_data, valid_data, saved=saved, show_progress=config['show_progress']
# )

# # model evaluation
# test_result = trainer.evaluate(test_data, load_best_model=saved, show_progress=config['show_progress'])

# logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
# logger.info(set_color('test result', 'yellow') + f': {test_result}')

# {
#     'best_valid_score': best_valid_score,
#     'valid_score_bigger': config['valid_metric_bigger'],
#     'best_valid_result': best_valid_result,
#     'test_result': test_result
# }

08 Sep 13:56    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = C:\Users\tanch\anaconda3\envs\fyp0\Lib\site-packages\recbole\config\../dataset_example/ml-100k
checkpoint_dir = saved/
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 100}
eval_step = 10
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [6, 2, 2]}, 'order': 'RO', 'group_by': 'user', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'NDCG']
topk = [10, 20]
valid_metric = NDCG@20
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITE

In [17]:
config


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = C:\Users\tanch\anaconda3\envs\fyp0\Lib\site-packages\recbole\config\../dataset_example/ml-100k
checkpoint_dir = saved/
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 10}
eval_step = 10
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [6, 2, 2]}, 'order': 'RO', 'group_by': 'user', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'NDCG']
topk = [10, 20]
valid_metric = NDCG@20
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RA

In [6]:
train_data.iid_field

'item_id'

In [8]:
dataset.inter_feat[train_data.iid_field]

tensor([ 584,  668,  299,  ...,  721,  205, 1256])

In [9]:
dataset[:10]

The batch_size of interaction: 10
    user_id, torch.Size([10]), cpu, torch.int64
    item_id, torch.Size([10]), cpu, torch.int64


# run 1
- run simplx, confirmed the following params to control: 
    - embedding_size: length of vector representing each user and item
    - margin: threshold to discrimate hard negative samples (in CCL)
    - negative_weight: balances effect of postive and negative samples on loss (CCL)
    - gamma: balances user and their history of interactions for the final user embedding
    - aggregator: aggregator for user's item history
    - history_len: number of positive interactions to use per user
    - neg_sampling: controls number of negative samples per user
    - reg_weight: regularize all weights
- it worked!
    - SimpleX indeed reaches convergence very fast as mentioned in the paper
    - SimpleX appears to be quite sensitive to hyperparameters and there's quite a few more hyperparameters compared to LightGCN
- we want to find out why this is happening and whether we can bring these to LightGCN

In [1]:
model = "SimpleX"
dataset = 'ml-100k'
config_file_list = ['test0.yaml']
config_dict = {
    "show_progress": False,
    "log_wandb":True,
    "eval_step": 10,
    "epochs": 50, 
    "eval_args": {"split": {"RS": [6,2,2]}},
    "valid_metric":"NDCG@20",
    "metrics":["Recall","NDCG"],
    "topk":[10,20],
    
    "embedding_size":32,
    "margin":0.8,
    "negative_weight":100,
    "gamma":0.6,
    "reg_weight":0.001,
    "aggregator":"mean",
    "history_len":10,
    "neg_sampling":{"uniform":100}
}
saved = False

In [2]:
"""
recbole.quick_start
########################
"""
import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color


# def run_recbole(model=None, dataset=None, config_file_list=None, config_dict=None, saved=True):

# configurations initialization
config = Config(model=model, dataset=dataset, config_file_list=config_file_list, config_dict=config_dict)
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)
logger = getLogger()

logger.info(config)

# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

# model loading and initialization
init_seed(config['seed'], config['reproducibility'])
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=saved, show_progress=config['show_progress']
)

# model evaluation
test_result = trainer.evaluate(test_data, load_best_model=saved, show_progress=config['show_progress'])

logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
logger.info(set_color('test result', 'yellow') + f': {test_result}')

{
    'best_valid_score': best_valid_score,
    'valid_score_bigger': config['valid_metric_bigger'],
    'best_valid_result': best_valid_result,
    'test_result': test_result
}

C:\Users\tanch\anaconda3\envs\fyp0\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
03 Sep 11:52    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = C:\Users\tanch\anaconda3\envs\fyp0\Lib\site-packages\recbole\config\../dataset_example/ml-100k
checkpoint_dir = saved/
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 100}
eval_step = 10
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [6, 2, 2]}, 'order': 'RO', 'group_by': 'user', 'mode': 'ful

03 Sep 11:53    INFO  epoch 0 training [time: 20.47s, train loss: 1242.0710]
03 Sep 11:53    INFO  epoch 1 training [time: 17.26s, train loss: 821.8578]
03 Sep 11:53    INFO  epoch 2 training [time: 17.08s, train loss: 800.3585]
03 Sep 11:54    INFO  epoch 3 training [time: 17.74s, train loss: 785.2280]
03 Sep 11:54    INFO  epoch 4 training [time: 17.30s, train loss: 775.0277]
03 Sep 11:54    INFO  epoch 5 training [time: 17.19s, train loss: 769.0105]
03 Sep 11:55    INFO  epoch 6 training [time: 17.80s, train loss: 763.7639]
03 Sep 11:55    INFO  epoch 7 training [time: 18.25s, train loss: 758.6409]
03 Sep 11:55    INFO  epoch 8 training [time: 17.26s, train loss: 756.0954]
03 Sep 11:55    INFO  epoch 9 training [time: 18.01s, train loss: 752.6732]
03 Sep 11:55    INFO  epoch 9 evaluating [time: 0.63s, valid_score: 0.230200]
03 Sep 11:55    INFO  valid result: 
recall@10 : 0.1391    recall@20 : 0.2242    ndcg@10 : 0.2106    ndcg@20 : 0.2302
03 Sep 11:56    INFO  epoch 10 training [ti

{'best_valid_score': 0.2302,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.1391),
              ('recall@20', 0.2242),
              ('ndcg@10', 0.2106),
              ('ndcg@20', 0.2302)]),
 'test_result': OrderedDict([('recall@10', 0.1322),
              ('recall@20', 0.2234),
              ('ndcg@10', 0.219),
              ('ndcg@20', 0.2353)])}

In [19]:
model.neg_seq_len, model.embedding_size, model.margin, model.negative_weight, model.gamma, model.reg_weight, model.aggregator, model.history_len

(200, 64, 0.7, 300, 0.6, 0.1, 'mean', 20)

In [1]:
from parse import *

In [3]:
parse_args("python parse.py --model simplex")

TypeError: parse_args() takes 0 positional arguments but 1 was given

In [4]:
parse = argparse.ArgumentParser(description="Choose model")

In [6]:
dir(parse)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_action_groups',
 '_actions',
 '_add_action',
 '_add_container_actions',
 '_check_conflict',
 '_check_value',
 '_defaults',
 '_get_args',
 '_get_formatter',
 '_get_handler',
 '_get_kwargs',
 '_get_nargs_pattern',
 '_get_option_tuples',
 '_get_optional_actions',
 '_get_optional_kwargs',
 '_get_positional_actions',
 '_get_positional_kwargs',
 '_get_value',
 '_get_values',
 '_handle_conflict_error',
 '_handle_conflict_resolve',
 '_has_negative_number_optionals',
 '_match_argument',
 '_match_arguments_partial',
 '_mutually_exclusive_groups',
 '_negative_number_matcher',
 '_option_string_actions',
 '_optionals',
 '_parse_know